## Parallelized Tokenizer
---

We find the most frequently appearing tokens in our large file. The function `top_n_tokens` returns a list of the n most frequently appearing tokens and their frequencies. This is the simple implementation in basic Python.

In [ ]:
from collections import Counter

def top_n_tokens(filename):
    # tokenizing file
    tokenized = []
    with open(filename) as f:
        for line in f:
            # tokenize, one line at a time
            '''
            REDACTED
            '''

    c = Counter(tokenized) # create instance of Counter class
    top_n_tokens_list = c.most_common(n) # find the n most frequent tokens
                                                 #   and the number of occurrences
    return top_n_tokens_list

---
The function `t_dict` returns a dictionary. The keys are tokens that follow the token `t` on more than one line. The values are the number of lines in which the pattern is observed. This is the simple implementation in basic Python.

In [ ]:
def t_dict(filename, t):
    dict_t = {}
    term = None # a temp variable for holding the tokens

    with open(filename) as f:
        for line in f:
            # tokenize, one line at a time
            t = simple_tokenize(line)

            for i in range((len(t) - 1)): # checking all but the last token on a line
                '''
                REDACTED
                '''

    dict_t = {k:v for k,v in dict_t.items() if v > 1}
      # keep only those key value pairs where the value (count) is greater than 1
    return dict_t

---
Analyzing the [pointwise mutual information (PMI)](http://en.wikipedia.org/wiki/Pointwise_mutual_information) of tokens.

This chunk analyzes how large this problem will be. Determine the number of *distinct* tokens and distinct token pairs that exist.

In [ ]:
import itertools

single_tokens = {}
token_pairs = {}

with open('_.txt') as f:
    for line in f:
        # tokenize, one line at a time
        t = simple_tokenize(line)

        unique_t = list(set(t)) # get just the unique tokens
        '''
        REDACTED
        '''

        pairs = list(itertools.combinations(unique_t,2)) # find all pairs of tokens

        pairs = [list(pairs[i]) for i in range(len(pairs))] # convert tuples to list
        pairs_copy = pairs[:] # make a shallow copy of the list of pairs

        '''
        REDACTED
        '''

        pairs = [tuple(pairs[i]) for i in range(len(pairs))] # convert pairs back to tuples to be hashable

        for pair in pairs:
            if pair not in token_pairs: # check whether the token pair already exists in the dictionary
                token_pairs[pair] = 1 # if not, add it to the dictionary

25975
1969760


---
Implement the token queries.

In [ ]:
# the log function for computing PMI
from math import log
import itertools

######################################################################

single_tokens = {}
token_pairs = {}
n_lines = 0

with open('_.txt') as f:
    for line in f:
        n_lines = n_lines + 1
        # tokenize, one line at a time
        t = simple_tokenize(line)
        unique_t = list(set(t)) # get just the unique tokens
        '''
        REDACTED
        '''

######################################################################

while True:
    q = input("Input: ")
    if len(q) == 0:
        break
    '''
    REDACTED
    '''
    if len(q_tokens) == 1:
        threshold = 0
        '''
        REDACTED
        '''
        if q_tokens[0] in single_tokens: # checking that the token exists
            n_x = single_tokens[q_tokens[0]] # lookup the number of occurrences

            tokens = list(single_tokens.keys()) # get all token keys in the dictionary
            valid_pairs = []

            for token in tokens:
              pair = [q_tokens[0], token] # create a pair from input and each token in dictionary
              if (token != q_tokens[0]) and (tuple(pair) in token_pairs): # check that the token is not equal to
                                                                          #   input and that the pair exists
                n_xy = token_pairs[tuple(pair)] # lookup number of occurences of pair
                if n_xy >= threshold:
                    valid_pairs.append(pair) # store pair if it occurs at least threshold many times

            top_5_pmis = []

            for i in range(0, 5):
                max_pmi = 0
                max_pmi_key = None
                max_pmi_n_xy = 0

                '''
                REDACTED
                '''

                else:
                    break # break out of loop if there are fewer than 5 valid pairs
                top_5_pmis.append([max_pmi_key, max_pmi, max_pmi_n_xy]) # store the token pair, it's pmi and n(x,y)

        else: # if the token doesn't exist
            n_x = 0 # set n(x) = 0
            top_5_pmis = [] # no pairs to consider

        '''
        REDACTED
        '''


    elif len(q_tokens) == 2:

        pair = [q_tokens[0], q_tokens[1]]

        if tuple(pair) in token_pairs: # check that the pair exists
          '''
          REDACTED
          '''

        else:  # if the pair doesn't exist
            n_xy = 0 # set n(x, y) = 0
            pmi = 0 # pmi(x, y) = 0

        '''
        REDACTED
        '''


Input 1 or 2 space-separated tokens (return to quit): the end
  n(the,end) = 157
  PMI(the,end) = 0.3505058356267105
Input 1 or 2 space-separated tokens (return to quit): end
Input a positive integer frequency threshold: 3
  n(end) = 353
  high PMI tokens with respect to end (threshold: 3):
    n(end,rope's) = 5,  PMI(end,rope's) = 2.5402124568157887
    n(end,upper) = 3,  PMI(end,upper) = 2.1722356715211943
    n(end,latter) = 7,  PMI(end,latter) = 2.1548615754517715
    n(end,begun) = 4,  PMI(end,begun) = 1.679874450244795
    n(end,world's) = 3,  PMI(end,world's) = 1.3183637071994323
Input 1 or 2 space-separated tokens (return to quit): 


---
We again count the number of *distinct* tokens but now use Spark to do so.

In [ ]:
def distinct_tokens():
    lines = sc.textFile("_.txt") # import file
    words = lines.flatMap(lambda x: simple_tokenize(x)).map(lambda word: (word, 1))
                                  # tokenize and create tuples with prelim count
    numdistinct = words.reduceByKey(lambda x, y: 1).values().reduce(lambda x, y: x + y)
                                  # extract 1s and take their sum
    return numdistinct


---
This Spark program counts the number of distinct token pairs as before.

In [ ]:
import itertools

# Returns the count of distinct pairs
def distinct_pairs():
    lines = sc.textFile("_.txt") # import file
    tokenized_lines = lines.map(lambda x: simple_tokenize(x)) # tokenize lines
    distinct_token_lines = tokenized_lines.map(lambda x: list(set(x))) # keep only
                                  # a distinct set of words

    # Returns all combinations of two words in a sentence
    def pair(x):
        '''
        REDACTED
        '''

    pairs = distinct_token_lines.flatMap(lambda x: pair(x)) # apply pair function and flatten
    pairs_reverse = pairs.map(lambda x: x[::-1]) # reversing all the pairs
    '''
    REDACTED
    '''
    numdistinct = pairs.reduceByKey(lambda x, y: 1).values().reduce(lambda x, y: x + y)
                                  # extract 1s and take their sum

    return numdistinct


---
Here we calculate PMIs for every distinct token and n highest-probability tokens in a data-parallel fashion.

In [ ]:
# Returns a list of the top n (probability, count, token) tuples, ordered by probability
def top_n_tokens_prob(n):
    lines = sc.textFile("_.txt") # import file
    tokenized_lines = lines.map(lambda x: simple_tokenize(x)).flatMap(lambda x: list(set(x)))
                                    # tokenize the lines, get unique tokens then flatten

    counts = tokenized_lines.map(lambda word: (word, 1)).reduceByKey(lambda x, y: x + y)
                                    # create tuples then take the sum of 1s
    numLines = lines.count() # the number of lines in the file

    probabilities = counts.mapValues(lambda x: x / numLines) # compute probability for each token

    tokens = probabilities.join(counts) # join probability and count on token key

    # Reorders the entries in x
    def reorder(x):
        '''
        REDACTED
        '''

    result = tokens.map(lambda x: reorder(x)).sortByKey(ascending=False) # reorder the
                                      # entries in the RDD

    # Collapses x into a list from a list of lists
    def collapse(x):
        '''
        REDACTED
        '''

    result = result.map(lambda x: collapse(x)).take(n) # collapse all the entries in the RDD
    return result


---
PMIs done by using Spark.

In [ ]:
import itertools
from math import log

# Returns a list of tuples with the following format:
'''
REDACTED
'''
def PMI(threshold):
    n_lines = sc.accumulator(0) # initialize accumulator for number of lines
    sc.textFile("_.txt").foreach(lambda x: n_lines.add(1)) # increment accumulator
                                    # for each line
    lines = sc.textFile("_.txt") # import file
    tokenized_lines = lines.map(lambda x: simple_tokenize(x)).map(lambda x: list(set(x)))
                                    # tokenize lines, keep only unique tokens
    words = lines.map(lambda x: simple_tokenize(x)).flatMap(lambda x: list(set(x)))
                                    # tokenize lines, keep unique tokens and flatten

    counts = words.map(lambda word: (word, 1)).reduceByKey(lambda x, y: x + y)
                                    # create tuples then take the sum of 1s
    counts = dict(counts.collect()) # turn counts into a dictionary for easy lookup
    counts = sc.broadcast(counts) # broadcast the dictionary

    # Returns all combinations of two words in a sentence
    def pair(x):
        '''
        REDACTED
        '''

    n_lines = n_lines.value

    pairs = tokenized_lines.flatMap(lambda x: pair(x)) # apply pair function and flatten
    pairs_reverse = pairs.map(lambda x: x[::-1]) # reversing all the pairs
    pairs = pairs.union(pairs_reverse).map(lambda x: tuple(x)).map(lambda word: (word, 1))
                                  # including the reversed pairs in the overall list
    coOccurrences = pairs.reduceByKey(lambda x, y: x + y).filter(lambda x: x[1] >= threshold)

    '''
    REDACTED
    '''

    return result.collect()


---
Take a threshold $T$ and a sample size $N$ and find co-occuring tokens, as well as PMI$(x,y)$ and more. Choosse $N$ different $x$'s, uniformly at random to output.

In [ ]:
from math import log

# Returns a list of samp_size tuples with the following format:
'''
REDACTED
'''

def PMI_one_tok(threshold, samp_size):
    n_lines = sc.accumulator(0) # initialize accumulator for number of lines
    sc.textFile("_.txt").foreach(lambda x: n_lines.add(1)) # increment accumulator
                                    # for each line
    lines = sc.textFile("_.txt") # import file
    tokenized_lines = lines.map(lambda x: simple_tokenize(x)).map(lambda x: list(set(x)))
                                    # tokenize lines, keep only unique tokens
    words = lines.map(lambda x: simple_tokenize(x)).flatMap(lambda x: list(set(x)))
                                    # tokenize lines, keep unique tokens and flatten

    counts = words.map(lambda word: (word, 1)).reduceByKey(lambda x, y: x + y)
                                    # create tuples then take the sum of 1s
    counts = dict(counts.collect()) # turn counts into a dictionary for easy lookup
    counts = sc.broadcast(counts) # broadcast the dictionary

    # Returns all combinations of two words in a sentence
    def pair(x):
        '''
        REDACTED
        '''

    n_lines = n_lines.value

    pairs = tokenized_lines.flatMap(lambda x: pair(x)) # apply pair function and flatten
    pairs_reverse = pairs.map(lambda x: x[::-1]) # reversing all the pairs
    pairs = pairs.union(pairs_reverse).map(lambda x: tuple(x)).map(lambda word: (word, 1))
                                    # including the reversed pairs in the overall list
    coOccurrences = pairs.reduceByKey(lambda x, y: x + y).filter(lambda x: x[1] >= threshold)

    # Appends b to a
    def appender(a,b):
        '''
        REDACTED
        '''

    # Extends a with b
    def extender(a, b):
        '''
        REDACTED
        '''

    coOccurrences = coOccurrences.map(lambda x: (x[0][0], ((x[0][0], x[0][1]), x[1])))
                                    # this format makes the first token the key
                                    # so that it may be combined on

    coOccurrences = coOccurrences.combineByKey(lambda x: [x],
                                               appender,
                                               extender).takeSample(False, samp_size)
                                    # combine the co-occuring tokens into one list
                                    # take a random sample of size N

    coOccurrences = sc.parallelize(coOccurrences) # turn back into an RDD

    # Creates the required format for the output
    def formatting(a):
        r = [] # initialize empty list
        for p in a:
          '''
          REDACTED
          '''
        return r

    result = coOccurrences.map(lambda y: (y[0], formatting(y[1])))
                                  # apply the formatting function to all entries in RDD

    return result.collect()
